In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parser
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
import pickle
import json

# Обработка основного датасета

In [ ]:
df = pd.read_csv('/Users/ula/Downloads/Merch_CB_hack.csv', error_bad_lines=False)
df = df.dropna()
df = df.reset_index()
df.isnull().sum()
df['day'] = pd.to_datetime(df['day'])
df = df.sort_values(by='day', ignore_index=True)
df

# Корреляция 

In [ ]:
# Test for seasonality
from pandas.plotting import autocorrelation_plot

# Draw Plot
plt.rcParams.update({'figure.figsize':(10,6), 'figure.dpi':120})
autocorrelation_plot(df['cashback'].tolist())

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Draw Plot
fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
plot_acf(df['cashback'].tolist(), lags=50, ax=axes[0])
plot_pacf(df['cashback'].tolist(), lags=50, ax=axes[1])

# Оценка стационарности через KPSS

In [ ]:
data = df['cashback']
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

kpss_test(data)

# Оценка стационарности через ADF

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(data, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'n_lags: {result[1]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')    

# Декомпозиция

In [ ]:
# Multiplicative Decomposition 
multiplicative_decomposition = seasonal_decompose(data["cashback"], model="multiplicative", period=6)

# Additive Decomposition
additive_decomposition = seasonal_decompose(data["cashback"], model="additive", period=6)

# M-tive + A-tive plots
plt.rcParams.update({'figure.figsize': (15,12)})
multiplicative_decomposition.plot().suptitle('Multiplicative Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

additive_decomposition.plot().suptitle('Additive Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

# Установка классовых датасетов

In [284]:
data = pd.read_csv('/Users/ula/Downloads/backup.csv')
data.head()

,Unnamed: 0,day,cashback
0,0,2022-01-21 00:00:00,6572.103187
1,1,2022-01-22 00:00:00,7453.472366
2,2,2022-01-23 00:00:00,5776.950000
3,3,2022-01-24 00:00:00,4333.526374
4,4,2022-01-25 00:00:00,4862.104737


In [267]:
# сделать для каждый категории свой датафрйем со средним чеком в день (аналогично data)
df_it = {}
df_store = {}
df_food = {}

# Главный код

### По функциям

In [5]:
categories = {'food': ['AGAMA', 'Delivery Club', 'Elementaree', 'GLOBUS', 'Greenbox Рационы', 'Grow Food', 'Growfood', 'JustFood', 'My Food', 'SPAR', 'Vprok.ru Перекрёсток', 'Wow Food', 'YAMDIET', 'Yamdiet', 'pobo', 'Ашан', 'Бахетле', 'Веселый Водовоз', 'ВкусВилл', 'ВкусВилл Готовит', 'ВкусВилл в Тинькофф', 'ВкусМил', 'ВкусМил от ВкусВилл', 'ВкусМилл от ВкусВилл', 'ВкусНаДом', 'Вкусвилл', 'Все в разделе «ВкусВилл»', 'Гипермаркет Карусель', 'Дикси', 'Доставка питания YAMDIET', 'Доставка питания Yamdiet', 'ЛЕНТА', 'ЛЕНТА Онлайн', 'Лавка Edoque', 'Лента', 'Лента Онлайн', 'Магнит', 'Магнолия', 'Мегамаркет', 'Михайлик Kitchen', 'О’КЕЙ', 'Пан Запекан', 'Перекрёсток', 'Петрович', 'Порядок', 'Порядок.ру', 'Правильная корзинка', 'ПриЕм!', 'Программы питания Level Kitchen', 'Пятёрочка', 'Сhef At Home', 'Самокат', 'Сахар', 'Сеть магазинов «Пятёрочка»', 'Слата', 'Ужин Дома', 'Шефмаркет', 'Яндекс Еда', 'Яндекс Лавка', 'Яндекс.Еда', 'Яндекс.Еда и Лавка', 'Яндекс.Лавка', 'Ярче', 'Ярче!', 'интернет-магазин GLOBUS', 'интернет-магазин ВкусВилл'],
              'education': ['Advance', 'GoPractice Growth', 'LingvaBOOM', 'MyFitlab', 'Puzzle English', 'SKYENG', 'School of Practical Investment', 'Skyeng', 'Skyeng Math', 'Skyeng Премиум', 'Skypro', 'Skysmart', 'Skysmart Premium', 'Smart Reading', 'SmartReading', 'Smartreading', 'Storytel', 'X10 Academy', 'Аудиомания', 'Аудиомания.ру', 'Для студентов ВШЭ', 'Для студентов МГТУ им. Баумана', 'Для студентов МГУ', 'Для студентов НИУ ВШЭ СПб', 'Интернет-магазин «Читай-город»', 'Интернет-магазин Читай-город', 'Испаника', 'Нетология', 'Онлайн школа  LingvaBOOM', 'Онлайн школа Испаника', 'Парусная Академия', 'Республика', 'Сотка', 'Тетрика', 'Умскул', 'Учи.ру', 'Фоксфорд', 'Читай город', 'Читай-город', 'Школа идеального тела'],
              'medicine': ['ADRIA', 'Alter', 'Aravia', 'Armani beauty', 'Delux thai spa', 'Diamarka', 'Doma Beauty Place', 'Elemis', 'Erborian', 'KIKO MILANO', 'KRASOTKAPRO.RU', 'Kiko Milano', 'LETIQUE', 'Lancome.ru', 'Los Brows', 'MIXIT', 'Medalp', 'Noerden', 'Pharmacosmetica', 'SEPHORA', 'Sisley-Paris', 'SmartMed', 'Socolor', 'Soda', 'Well Clinic', 'Yahmur Space', 'Yves Rocher', 'biblioteka aromatov', 'imkosmetik', 'Акушерство.ru', 'Аптека Вита', 'Аптека Низких Цен', 'Библиотека ароматов', 'Галерея косметики', 'Горздрав', 'Государственная Аптека', 'Дентал Бутик', 'ДиаМарка', 'ЕМС', 'Европейский Медицинский Центр', 'ЗдравСити', 'Интернет-бутик «YVES ROCHER»', 'Интернет-магазин Акушерство.ру', 'КрасоткаПро', 'Красоткапро', "Л'Этуаль", 'Лаборатория ДНКОМ', 'Линзмастер', 'Лошадиная Сила', 'Лошадиная сила', 'МЕДСИ', 'МЕДСИ сеть клиник', 'Магнит косметик', 'Медси', 'Миссис Лазер', 'Моя аптека', 'Народная стоматология', 'ОРТЕКА', 'Оазис-спа', 'Огни Олимпа', 'Оптика Точка зрения', 'Ортека', 'Офтальмологическая клиника Спектр', 'Планета Здоровья', 'Сеть аптек «Моя аптека»', 'Сеть аптек Моя аптека', 'Сиблабсервис', 'Телемедицина от МЕДСИ', 'Телемедицина от МЕДСИ ', 'Точка зрения', 'УЗИ студия', 'Улыбка радуги', 'ЦЭЛТ', 'Четыре глаза', 'интернет-бутик Yves Rocher', 'интернет-бутик «YVES ROCHER»'],
              'travel': ['4SEASONS', 'Art Deco Primorsky', 'Art Nuvo Palace', 'Bronevik', 'Bronevik.com', 'Delta Sirius', 'FUN&SUN', 'Grand Wellness', 'Level Travel', 'Level.Travel', 'Level.Travel Туры', 'METRO Cash&Carry', 'Ostrovok.ru', 'QuickMADE', 'S7', 'S7 Airlines', 'SHELL', 'Shell', 'TUI', 'TUI I FUN&SUN', 'Tvil.ru', 'UBER', 'Uber Russia', 'Urent', 'Whoosh', 'tutu Отели', 'Автотека', 'Аква-Вита', 'Аква-Стайл', 'Бизнес-залы РЖД', 'ВелоСклад', 'Делимобиль', 'Купибилет', 'Лукойл', 'Манжерок', 'Манжерок ', 'Петровский Арт Лофт', 'РАМК', 'Русский Авто-Мото Клуб', 'Русский АвтоМотоКлуб (РАМК)', 'СИТИДРАЙВ', 'СЛЕТАТЬ.РУ', 'Сеть автомоек 4SEASONS', 'Ситидрайв', 'Ситимобил', 'Слетать.Ру', 'Спутник', 'Суточно.ру', 'ТВИЛ', 'Такси Maxim', 'Тинькофф Отели', 'Туту Отели', 'Экспресс Точка Ру', 'Юрент', 'Яндекс Такси', 'Яндекс.Такси'],
              'purchase': ['#SEKTA', '1С Интерес', '2MOOD', '585 Золотой', '585*Золотой', 'ATAK', 'AliExpress', 'AllTime', 'AllTime.ru', 'Bluesleep', 'Bonafide', 'Bonafide ', 'CERAMISU', 'COLINS', 'Cheese It', 'Code4game', 'Consul', 'Cеть хобби-гипермаркетов «Леонардо»', 'DKNY', 'Designboom', 'Dr.Head', 'ESET NOD32', 'ESETNOD32', 'Etam', 'FARFETCH', 'Familia', 'Farfetch', 'Fissman', 'Fix Price', 'FloraExpress', 'Futuriqa', 'GFN', 'GRASS', 'Giox', 'Goods', 'Goods.ru', 'Grass', 'Gulliver market', 'HANDWERS', 'Handwers', 'Happywear', 'I am studio', 'ISTNOVA', 'IVI', 'IVI в приложении Тинькофф', 'IVI от Тинькофф', 'Intelinvest', 'Intelinvest ', 'Juicy Couture', 'KANZLER', 'KARI', 'KARI KIDS', 'Kanzler', 'Karcher', 'Kari', 'Kidzania', 'Kraft Flowers', 'Lamoda', 'Lassie', 'Lee', 'MATE flowers', 'MEGOGO', 'MIUZ Diamonds', 'Marc & Andre', 'Marks & Spencer', 'Masar Lingerie', 'Megogo', 'Mon Bon', 'Movavi', 'My Hygge Box', 'NEBBIA', 'Nappyclub', 'OZON', 'OZON.ru', 'Olympus', 'Ozon', 'Ozon.ru', 'PETKIT', 'PREMIER', 'PRO.FINANSY', 'Petshop.ru', 'Petstory', 'Petstory ', 'Reima', 'Respect', 'Rondell', 'Routemark', 'SENAT', 'SOLAR', 'SUPERPET', 'Samsung', 'Seiko Club', 'Shopping live', 'Simple', 'Softline Store', 'SuperStep', 'Svetlov', 'TEZENIS', 'Teboil', 'Tkano', 'VK Музыка', 'Wink', 'World of watch', 'Yves Saint Laurent', 'Zolla', 'adidas', 'e2e4', 'goods.ru', 'ivi', 'kari', 'kari KIDS', 'more.tv', '«Лаборатория Касперского»', 'Аленка', 'Арбатский БаЗар', 'Бетховен', 'Братья Чистовы', 'Бронницкий ювелир', 'Верный', 'Ветеринария онлайн Petstory', 'Ветеринарный центр', 'Газпромнефть', 'Горбилет', 'Деловые Линии', 'Детский Мир', 'Динозаврик', 'Доставка цветов FloraExpress', 'Зелёная лиса', 'Золотое Яблоко', 'Зоомагазин Динозаврик', 'ИВИ', 'Иви в приложении Тинькофф', 'Игры в Тинькофф Городе', 'Интернет - магазин SEPHORA', 'Интернет-магазин 585*Золотой', 'Интернет-магазин Lancome.ru', 'Интернет-магазин SuperStep', 'Интернет-магазин TEZENIS', 'Интернет-магазин «Технопарк»', 'Интернет-магазин Динозаврик', 'Интернет-магазин Технопарк', 'КОД4ГЕЙМ', 'Карат', 'Каток на Кремлёвской набережной', 'Кидзания', 'Кораблик', "Л'Окситан", 'Лаборатория Касперского', 'Леонардо', 'Логомашина', 'МегаФон', 'Мираторг', 'Мокрый нос', 'Монетка', 'Московский ювелирный завод', 'Николаевский', 'Опека', 'ПЕРЕСТРОЙКА', 'Плюс Мульти', 'Полимакс', 'РИВ ГОШ', 'Рив Гош', 'Роснефть', 'Ростелеком', 'Русский остров', 'СЕТЬ СВЯЗНОЙ', 'СТРОЙПЛАТФОРМА', 'СберМаркет', 'СберМегаМаркет', 'Связной', 'Спектр', 'Столото', 'Строительный двор', 'Строительный двор Рассрочка', 'Строительный двор Таргет', 'ТВОЕ', 'Татнефть', 'Технопарк', 'Тинькофф Игры', 'Тинькофф Страхование', 'Тинькофф каток', 'Топливо в Тинькофф', 'Топливо в Тинькофф (FUEL)', 'Тортомастер', 'Точка любви', 'Хобби-гипермаркет Леонардо', 'Хобби-гипермаркеты «Леонардо»', 'Хобби-гипермаркеты Леонардо', 'ЦВЕТЫ НА РАЙОНЕ', 'ЦСКА', 'Цветов.ру', 'Цветы на районе', 'ЧЕТЫРЕ ГЛАЗА', 'Четыре Лапы', 'Чистая линия', 'Чёрное Озеро', 'Яндекс 360', 'Яндекс Заправки', 'Яндекс Плюс', 'Яндекс.Заправки', 'Яркий Фотомаркет', 'Яркий фотомаркет', 'интернет-магазин SuperStep', 'интернет-магазин TEZENIS', 'интернет-магазин Технопарк'],
              'restaurant': ['BB&BURGERS', 'BURGER KING', 'Burger King', 'Cofix', "Domino's Pizza", 'Four', 'Gent', 'KFC', 'Maestrello', 'PIZZASUSHIWOK.RU', 'PhoBo', 'Pizza Maestrello', 'PizzaSushiWok', 'PrimeMeat', 'PrimeMeat.ru', 'Tasty Coffee', 'Бирвайн', 'Бургер Кинг', 'ВьетКафе', 'Гурманика', 'ДОДО Пицца', 'Додо', 'Додо Пицца', 'Кухня на районе', 'НИЯМА', 'Рестораны', 'Сушкоф и Дель Песто', 'Сушкоф и пицца', 'Теремок', 'Теремок ', 'Теремок Спб'],
              'bigpurchase': ['AUTODOC.RU', 'Anytime Prime', 'Braun', 'DNS', 'Don Plafon', 'Fandeco', 'Garlyn', 'Greenbox', 'HOBOT', 'HOFF', 'Haier', 'Inoxtime', 'LAZURIT', 'Lapsi', 'Lavita', 'Leran', 'Level Kitchen', 'Lustron', 'Lustron.ru', 'MELEON', 'Maxwell', 'Pushe', 'SantPrice.ru', 'TESSER', 'TESSER.RU', 'Xcom-shop', 're:Store', 'restore:', 'Гардиан', 'Двери ГАРДИАН', 'Двери Гардиан', 'Дивайн Лайт', 'ЛЮ.ру', 'Леруа Мерлен', 'М.Видео', 'Маркет света', 'Сантехника-Рум', 'Ситилинк', 'Фиссмания', 'Холодильник.ру', 'Эльдорадо']}

In [288]:
def which_category(name: str): # -> str
    for key_cat, val_cat in categories.items():
        if name in val_cat:
            return key_cat
    return 'ErrorCategoryQualifier: this company not in Data'

def create_full_dict(name: str, budget: int, category: str, date_end): # -> dict
    d_full = {name: [category, budget, date_end, 0, False]}
    return d_full

def determinant_model(category): # УЧЕСТЬ ЧТО НАЗВАНИЕ ПОЛНОЕ
    name_model = f'model_{category}.pkl'
    with open(name_model, 'rb') as pkl:
            model = pickle.load(pkl)
    return model

def get_predict_old(model, date_end): # -> DataFrame     ПРОВЕРИТЬ РАБОТОСПОСОБНОСТЬ 
    periods = (date_end - pd.to_datetime('2023-01-01')).dt.days
    predict = model.predict(n_periods=periods)
    dates = pd.date_range(pd.to_datetime('2023-01-01'), date_end)
    predict = predict.to_frame(name='pred_cash')
    df_pred = pd.concat([dates, predict], axis=1)
    return df_pred

def change_datatime_action(predicted_df, full_df):  # -> DataFrame   
    for i in range(predicted_df.shape[0]):
        spent_now = change_spent_budget(predicted_df, full_df, i)[0][-2]
        if spent_now >= full_df[0][1]:
            full_df[0][-1] = True 
            full_df[0][2] = pd.to_datetime(predicted_df.iloc[i, 0])
    return full_df

def change_spent_budget(predicted_df, full_df, index):   # -> DataFrame 
    full_df[0][-2] = full_df[0][-2] + predicted_df.iloc[index, -1]
    return full_df

In [ ]:
def get_predict(name, budget, date_end):

    with open('predictions.json', 'r') as json_file:
        json_data = json.load(json_file)

    for key_cat, val_cat in categories.items():
        if name in val_cat:
            category = key_cat
        
    full_df = {name: json_data[name]}

    name_model = f'model_{category}.pkl'
    with open(name_model, 'rb') as pkl:
            model = pickle.load(pkl)

    periods = (date_end - pd.to_datetime('2023-01-01')).dt.days
    predict = model.predict(n_periods=periods)

    dates = pd.date_range(pd.to_datetime('2023-01-01'), date_end)
    predict = predict.to_frame(name='pred_cash')
    df_pred = pd.concat([dates, predict], axis=1)
    
    sum_cash = 0
    for i in range(df_pred.shape[0]):
        sum_cash += df_pred.iloc[i, -1] 
        if sum_cash >= budget:
            full_df[name] = pd.to_datetime(df_pred.iloc[i, 0])
    
    json_data[name] = full_df[name]
    with open('predictions.json', 'w') as f:    
        json.dump(json_data, f)

# Обучение модели, подбор параметров, предсказание

In [ ]:
df = pd.read_csv('/Users/ula/Downloads/backup.csv')

model = pm.auto_arima(df["cashback"], start_p=1, start_q=1,
                         test='kpss',
                         max_p=5, max_q=5, m=6,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

predict, confint = model.predict(n_periods=5, return_conf_int=True)
print(predict)

In [275]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                  396
Model:             SARIMAX(1, 0, 0)x(0, 1, [1, 2], 6)   Log Likelihood               -3638.588
Date:                                Sat, 24 Feb 2024   AIC                           7287.176
Time:                                        23:23:53   BIC                           7307.007
Sample:                                             0   HQIC                          7295.037
                                                - 396                                         
Covariance Type:                                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     26.9708     21.465      1.257      0.209     -15.099      69.041
ar.L1          0.7582      0.017     45.315      0.000       0.725       0.791
ma.S.L6       -0.6831      0.026    -26.643      0.000      -0.733      -0.633
ma.S.L12      -0.2270      0.036     -6.289      0.000      -0.298      -0.156
sigma2      7.291e+06   2.36e+05     30.861      0.000    6.83e+06    7.75e+06
===================================================================================
Ljung-Box (L1) (Q):                   1.09   Jarque-Bera (JB):              5782.71
Prob(Q):                              0.30   Prob(JB):                         0.00
Heteroskedasticity (H):               8.01   Skew:                            -0.21
Prob(H) (two-sided):                  0.00   Kurtosis:                        21.86
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
'''
version 1:

max_p=5, max_q=5, m=6, d=2

ARIMA(5,2,0)(2,1,0)[6] 
'''

'''
version 2:

max_p=5, max_q=5, m=7, d=2

ARIMA(4,2,1)(1,1,1)[7]  
'''
